In [ ]:
import glob
import pickle

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

%matplotlib inline

# Transform

In [ ]:
def draw_lines(img, points, color=[255, 0, 0], thickness=2):
    pt1 = points[0]
    pt2 = points[1]
    pt3 = points[2]
    pt4 = points[3]

    cv2.line(img, (pt1[0], pt1[1]), (pt2[0], pt2[1]), color, thickness)
    cv2.line(img, (pt2[0], pt2[1]), (pt3[0], pt3[1]), color, thickness)
    cv2.line(img, (pt3[0], pt3[1]), (pt4[0], pt4[1]), color, thickness)
    cv2.line(img, (pt4[0], pt4[1]), (pt1[0], pt1[1]), color, thickness)

In [ ]:
# x2-x1/y2-y1 = x2-x3/y2-y3
def get_x3(x1, x2, y1, y2, y3):
    x3 = x2-(((x2-x1)/(y2-y1))*(y2-y3))
    print(x3)
    return x3

In [ ]:
images = glob.glob('test_images/straight_lines*.jpg')

transform_pickle_path = 'wide_transform_pickle.p'

dist_pickle_path = 'wide_dist_pickle.p'
with open(dist_pickle_path, mode='rb') as f:
    dist_pickle = pickle.load(f)
    
mtx, dist = dist_pickle['mtx'], dist_pickle['dist']

y1 = 700 # 720, 650
y2 = 465
y3 = 460

left_x1 = 220 # 200, 300
left_x2 = 573 
left_x3 = get_x3(left_x1, left_x2, y1, y2, y3)

right_x1 = 1060 # 1100, 1000
right_x2 = 710
right_x3 = get_x3(right_x1, right_x2, y1, y2, y3)

src_pt1 = [left_x3, y3]
src_pt2 = [right_x3, y3]
src_pt3 = [right_x1, y1]
src_pt4 = [left_x1, y1]
src = np.float32([src_pt1, src_pt2, src_pt3, src_pt4])

dst_pt1 = [300, 0]
dst_pt2 = [1000, 0]
dst_pt3 = [1000, 720]
dst_pt4 = [300, 720]
dst = np.float32([dst_pt1, dst_pt2, dst_pt3, dst_pt4])

In [ ]:
# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 6))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(img)
    sub1.set_xticks(())
    sub1.set_yticks(())
    
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    draw_lines(img, src)
    sub2.imshow(img)
    sub2.set_xticks(())
    sub2.set_yticks(())

fig.tight_layout()
plt.show()

In [ ]:
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)

# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 6))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    img_size = img.shape[1::-1]
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    draw_lines(img, src)
    sub1.imshow(img)
    if idx == 0:
        sub1.set_title('Original Images with source points')

    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    draw_lines(warped, dst)
    sub2.imshow(warped)
    if idx == 0:
        sub2.set_title('Transformed Images with destination points')

fig.tight_layout()
plt.show()

In [ ]:
images = glob.glob('test_images/test*.jpg')

fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(img, cmap='gray')
    sub1.set_xticks(())
    sub1.set_yticks(())

    # Draw and display the undistorted images
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(warped, cmap='gray')
    sub2.set_xticks(())
    sub2.set_yticks(())

fig.tight_layout()
plt.show()

# Threshold

In [ ]:
# Edit this function to create your own pipeline.
def pipeline(img, l_thresh=(170, 180), sl_thresh=(70, 250), s_thresh=(175, 250), sx_thresh=(20, 100), gray=True):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1]) & (s_channel >= sl_thresh[0]) & (s_channel <= sl_thresh[1])] = 1

    if gray == True:
        binary_output = np.zeros_like(sxbinary)
        binary_output[(l_binary == 1) | (s_binary == 1) | (sxbinary == 1)] = 1
        return binary_output
    else:
        # Stack each channel
        # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
        # be beneficial to replace this channel with something else.
        color_binary = np.dstack(( l_binary, sxbinary, s_binary))
        return color_binary

In [ ]:
# Step through the list and search for chessboard corners
fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]

    # Draw and display the original images
    wraped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(wraped)
    sub1.set_xticks(())
    sub1.set_yticks(())

    # Apply each of the thresholding functions  
    wraped_binary = pipeline(img, gray=False)
    wraped_binary = cv2.warpPerspective(wraped_binary, M, img_size, flags=cv2.INTER_LINEAR)
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(wraped_binary, cmap='gray')
    sub2.set_xticks(())
    sub2.set_yticks(())


fig.tight_layout()
plt.show()

# Sliding Window Search

In [ ]:
def get_slided(binary_warped, left_fit = None, right_fit = None):
    
    if left_fit == None and right_fit == None:
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 70
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    else:
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)


    return result, left_fitx, right_fitx, ploty, left_fit, right_fit

In [ ]:
def get_curverad_text(ploty, leftx, rightx):
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])    
   
    return 'LC: ' + str(round(left_curverad, 2)) + 'm, RC: ' + str(round(right_curverad, 2)) + 'm'

In [ ]:
fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]
    img = pipeline(img)
    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(binary_warped, cmap='gray')
    sub1.set_xticks(())
    sub1.set_yticks(())

    slided, left_fitx, right_fitx, ploty, left_fit, right_fit = get_slided(binary_warped)
    curverad_text = get_curverad_text(ploty, left_fitx, right_fitx)
    
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(slided)
    sub2.plot(left_fitx, ploty, color='yellow')
    sub2.plot(right_fitx, ploty, color='yellow')
    sub2.set_xlim([0, 1280])
    sub2.set_ylim([720, 0])
    sub2.set_xticks(())
    sub2.set_yticks(())
    sub2.text(100, 100, curverad_text, bbox={'facecolor':'yellow', 'alpha':0.5, 'pad':10})


fig.tight_layout()
plt.show()

# Drawing

In [ ]:
fig = plt.figure(figsize=(10, 15))

for idx, fname in enumerate(images):    
    img = mpimg.imread(fname)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    img_size = img.shape[1::-1]
    img = pipeline(undist)
    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    slided, left_fitx, right_fitx, ploty, left_fit, right_fit = get_slided(binary_warped)
    curverad_text = get_curverad_text(ploty, left_fitx, right_fitx)

    # Draw and display the original images
    sub1 = plt.subplot(len(images), 2, (idx*2)+1)
    sub1.imshow(slided)
    sub1.plot(left_fitx, ploty, color='yellow')
    sub1.plot(right_fitx, ploty, color='yellow')
    sub1.set_xlim([0, 1280])
    sub1.set_ylim([720, 0])
    sub1.set_xticks(())
    sub1.set_yticks(())
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    result = cv2.putText(result, curverad_text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255), 4)
    sub2 = plt.subplot(len(images), 2, (idx*2)+2)
    sub2.imshow(result)
    sub2.set_xticks(())
    sub2.set_yticks(())


fig.tight_layout()
plt.show()

# Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
class Lane_lines():
    def __init__(self):
        self.left_fit = None
        self.right_fit = None

In [ ]:
lane_lines_stack = []

def process_image(image):
    previous_left_fit = None
    previous_right_fit = None
    if (len(lane_lines_stack) > 0):
        previous_lane_line = lane_lines_stack.pop()
        previous_left_fit = previous_lane_line.left_fit
        previous_right_fit = previous_lane_line.right_fit
    
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    img_size = image.shape[1::-1]
    img = pipeline(undist)
    binary_warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    slided, left_fitx, right_fitx, ploty, left_fit, right_fit = get_slided(binary_warped, previous_left_fit, previous_right_fit)
    curverad_text = get_curverad_text(ploty, left_fitx, right_fitx)
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    result = cv2.putText(result, curverad_text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255), 4)

    lane_lines = Lane_lines()
    lane_lines.left_fit = left_fit
    lane_lines.right_fit = right_fit
    lane_lines_stack.append(lane_lines)
    
    return result

In [ ]:
output_project_video = 'output_project_video.mp4'
project_video = VideoFileClip("project_video.mp4")
processed_project_video = project_video.fl_image(process_image) #NOTE: this function expects color images!!
%time processed_project_video.write_videofile(output_project_video, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_project_video))

In [ ]:
output_challenge_video = 'output_challenge_video.mp4'
challenge_video = VideoFileClip("challenge_video.mp4")
processed_challenge_video = challenge_video.fl_image(process_image) #NOTE: this function expects color images!!
%time processed_challenge_video.write_videofile(output_challenge_video, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_challenge_video))

In [ ]:
output_harder_challenge_video = 'output_harder_challenge_video.mp4'
harder_challenge_video = VideoFileClip("harder_challenge_video.mp4")
processed_harder_challenge_video = harder_challenge_video.fl_image(process_image) #NOTE: this function expects color images!!
%time processed_harder_challenge_video.write_videofile(output_harder_challenge_video, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_harder_challenge_video))